CDS 486: Homework 3

Matthew Miller

02/19/19

### Importing Packages

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import geopandas
import geoplot
import geoplot.crs as gcrs
from shapely.geometry import Point, Polygon
import math
'''
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
'''
plt.style.use('dark_background')

### Import Data

In [2]:
CC_Data = pd.read_csv("data/Dataset1_combined.csv")

In [3]:
CC_Data.head()

,timestamp,latitude,longitude,altitude,speedmph,speedkmh,accuracy
0,12/22/2018 12:09:32 PM,41.603367,-86.754688,259,48.5,78.0,15.975934
1,12/22/2018 12:09:34 PM,41.603271,-86.755283,266,52.0,83.7,10.000000
2,12/22/2018 12:09:35 PM,41.603228,-86.755552,265,51.3,82.6,10.000000
3,12/22/2018 12:09:37 PM,41.603144,-86.756071,261,50.9,81.9,10.000000
4,12/22/2018 12:09:39 PM,41.603064,-86.756573,266,52.0,83.7,10.000000


In [6]:
'''
#Tidy the timestamp data
time_list = []
for i in CC_Data["timestamp"]:
    ''This loop separates the clock time from the rest of the string''
    start = 0 #index for slicing
    stop = 0 #index for slicing
    flag = 0
    for j in range(len(i)):
        if (i[j] == " " and flag == 0):
            start = j+1
            flag = 1
        elif (i[j] == " " and flag == 1):
            stop = j
    time_list.append(i[start:stop])

CC_Data["time"] = time_list



second_list = []
for time in CC_Data["time"]:
    'this will create a column of all the times in seconds....this is easier to work with''
    x = time.split(":")
    hour = int(x[0])
    minute = int(x[1])
    second = int(x[2])
    second_list.append((hour%12)*60^2 + minute*60 + second)

for i in range(len(second_list)):
    'This changes the list so that each time is relative to the start time''
    second_list[i] -= 574

CC_Data["seconds"] = second_list
''';

In [14]:
#Tidy the timestamp data
time_list = []
for i in CC_Data["timestamp"]:
    '''This loop separates the clock time from the rest of the string'''
    start = 0 #index for slicing
    stop = 0 #index for slicing
    flag = 0
    for j in range(len(i)):
        if (i[j] == " " and flag == 0):
            start = j+1
            flag = 1
        elif (i[j] == " " and flag == 1):
            stop = j
    time_list.append(i[start:stop])

CC_Data["time"] = time_list

####################################

second_list = []
for i in range(len(CC_Data)):
    '''this will create a column of all the times in seconds, relative to the start time....this is easier to work with'''
    if (i == 0):
        x = CC_Data["time"].iloc[i]
        x = x.split(":")
        prev_hour = int(x[0])
        prev_minute = int(x[1])
        prev_second = int(x[2])
        prev_time = 0 #previous value stored since init
        second_list.append(0)
    else:
        x = CC_Data["time"].iloc[i]
        x = x.split(":")
        hour = int(x[0])
        minute = int(x[1])
        second = int(x[2])
        
        if ((prev_hour == hour) and (prev_minute == minute)):
            #if the hour and minute are the same
            sec = second - prev_second #finds how many seconds since previous instance
            second_list.append(prev_time + sec)
            prev_time += sec
        elif ((prev_hour == hour) and (prev_minute != minute)):
            #if the hour is the same, but different minutes
            sec = (second + (60-prev_second)) + ((minute-prev_minute-1) * 60) #finds how many seconds since previous instance
            #the ((minute-prev_minute-1) * 60) is there in case the jump is larger than a minute
            second_list.append(prev_time + sec)
            prev_time += sec       
        else:
            #if the hour has changed -- assuming the change is never more than 1 hour
            sec = (minute*60 + second)
            second_list.append(prev_time + sec)
            prev_time += sec
        prev_hour = hour
        prev_minute = minute
        prev_second = second

In [15]:
len(CC_Data)

361355

In [16]:
len(second_list)

361355

In [17]:
CC_Data["seconds"] = second_list

In [18]:
#remove points where the accuracy >10
CC_Data = CC_Data[CC_Data["accuracy"] <= 10]

In [19]:
CC_Data.head()

,timestamp,latitude,longitude,altitude,speedmph,speedkmh,accuracy,time,seconds
1,12/22/2018 12:09:34 PM,41.603271,-86.755283,266,52.0,83.7,10.000000,12:09:34,2
2,12/22/2018 12:09:35 PM,41.603228,-86.755552,265,51.3,82.6,10.000000,12:09:35,3
3,12/22/2018 12:09:37 PM,41.603144,-86.756071,261,50.9,81.9,10.000000,12:09:37,5
4,12/22/2018 12:09:39 PM,41.603064,-86.756573,266,52.0,83.7,10.000000,12:09:39,7
9,12/22/2018 12:09:49 PM,41.602614,-86.759369,259,53.9,86.8,9.461421,12:09:49,17


#### Problem 1

How long is the combined journey, in miles traveled, from beginning
through end?

In [20]:
def Haversine(lat1, lon1, lat2, lon2): 
    R = 6372.8 # Earth radius in kilometers
 
    dLat = np.radians(lat2 - lat1)
    dLon = np.radians(lon2 - lon1)
    
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
 
    a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
 
    return R * c

In [21]:
Total_Dist = 0.0
for i in range(len(CC_Data)-1):
    Total_Dist += Haversine(lat1=CC_Data["latitude"].iloc[i], lon1=CC_Data["longitude"].iloc[i], lat2=CC_Data["latitude"].iloc[i+1], lon2=CC_Data["longitude"].iloc[i+1])

print("The total distance covered in this Cross Country was %0.3f Km or %0.3f miles" %(Total_Dist, Total_Dist*0.621371))

The total distance covered in this Cross Country was 10340.166 Km or 6425.079 miles


Plot the entire journey on a map. Please make your visual informative:
Don’t plot the data at a very high level of resolution (e.g., northern
hemisphere) such that the entire trip appears like a very short line
segment! Instead, use a bit lower resolution, so that the trip can be
followed (for example, on a map of the United States).

Which two major U.S. cities were visited on this trip? (NOTE: The
two major U.S. cities visited on this trip were those cities in which the
travelers spent more than 10 hours.) Can you identify the hotels in
which the traveler’s stayed, in each city? To do this you must understand what it might mean–from the perspective of GPS coordinates–to
spend time in a hotel. For example: What is the likely recorded speed
when someone is sleeping? There are other ways to find specific locations, but hotels are the easiest if we define “sleeping” in GPS terms.

In [73]:
##speed should be zero for "n" seconds
def GetStops(n = 10, data = CC_Data):
    '''This function will gather a list of latitude/longitude pairs where each pair represents a trainstop. A trainstop is defined as stopping for at least n minutes'''
    Stops = []
    #Stops = [[index in CC_data,"lat", "lon", "duration (minutes)"]]
    n = n*60 #time, in seconds of how long a stop is
    start = 0 #index  the last stop started
    stop = 0 #index the last stop stopped

    for i in (range(0,len(data)-1)):
        #for each element in the data
        if ((data["speedmph"].iloc[i] == 0) and (i >= stop)):
        #if (speed at location i is zero) and (we are after the last stop ended)
            start = i #index of stop's start time
            
            
            j = i+1 #start at next index to continue check
            while ((data["speedmph"].iloc[j] == 0)):
            #while the speed remains 0
                   stop = j
                   j += 1
          
            if ((data["seconds"].iloc[stop] - data["seconds"].iloc[start]) >= n):
            #if (the duration of the stop is long enough)
                Stops.append([i, data["latitude"].iloc[i], data["longitude"].iloc[i], (data["seconds"].iloc[stop] - data["seconds"].iloc[start])/60])
                
            
    return Stops

In [76]:
print(GetStops(n = 600)) #10 hours

[[92757, 47.608964, -122.33316299999998, 649.7]]


In [75]:
print(GetStops(n = 200))

[[92757, 47.608964, -122.33316299999998, 649.7], [139309, 37.805606, -122.416288, 236.18333333333334]]


The first major city visited was Seattle, Washington. The other was San Francisco, California. 

Hotel in Seattle: Hilton Seattle

Hotel in San Francisco: Pier 2620 Hotel  https://www.google.com/maps/place/37%C2%B048'20.2%22N+122%C2%B024'58.6%22W/@37.8056309,-122.4161343,18.91z/data=!4m5!3m4!1s0x0:0x0!8m2!3d37.805606!4d-122.416288

How many total stops were made on this trip, and what are the lat/lon
coordinates of each of these stops? (NOTE: Answering this question
will require you to define a stop. That definition may or may not be
statistical in nature. If you adopt a statistical definition, then you
should provide that definition and a short discussion–a few sentences
at most–defending your choice. Also note that “stops” were not made
in the two major cities. Instead, “stops” were made in transit to the
major cities, and also between them.)

In [30]:
S = GetStops()
len(S)

37

In [35]:
S1 = GetStops(n = 5)

In [33]:
S2 = GetStops(n = 200)
len(S2)

2

In [34]:
S2

[[92757, 47.608964, -122.33316299999998, 649.7],
 [139309, 37.805606, -122.416288, 236.18333333333334]]

Defining a stop as a 10 minute period of maintaining a speed of 0, there were 37 stops on this trip.

However, if we define a stop as 5 minutes, there were 75 stops on this trip

Plot the stop locations on a map.

Report the mean speed between stops, and the standard deviation of
the speed between stops.

In [45]:
Stops = GetStops(n = 5)

In [77]:
s_counter = 0 #this will be used to index through the Stops
avg = 0 #this will compute the running average
std = 0 #compute running std
avg_counter = 0 #counter for the mean
speed_data = [] #this will hold the mean speed and standard deviation of speeds between each stop
for i in range(1,len(CC_Data)):
    if (s_counter == len(Stops)):
        S = CC_Data["seconds"].iloc[-1] + 1
    else:
        S = Stops[s_counter][0]
    #The if else above enables us to get the mean speeds from the last stop onward, if need be
        
        
    if (i < S):
        avg_counter += 1
        old_avg = avg
        avg = ((avg_counter-1)*avg + CC_Data["speedmph"].iloc[i-1])/avg_counter
        std = (std + ((CC_Data["speedmph"].iloc[i-1]-avg)*(CC_Data["speedmph"].iloc[i-1]-old_avg)))
    else:
        speed_data.append([avg,std])
        avg_counter = 0
        avg = 0
        std = 0
        s_counter += 1

In [78]:
speed_data

[[41.61644094488196, 1212986.2817826741],
 [58.99483782549103, 2014743.6916674303],
 [50.3302389705882, 2135026.6902757366],
 [51.92268089053795, 4281153.001808899],
 [37.94750415973379, 848141.4575124786],
 [48.11449983708047, 2548590.804757255],
 [35.423455759599406, 3829717.3726794715],
 [60.05702323634082, 3313276.9468704103],
 [17.19265033407572, 321998.8114922052],
 [52.92181335356591, 2032193.4157321774],
 [62.29199554423909, 3663393.3073758753],
 [52.06594975256934, 4866988.918410745],
 [63.829026763990406, 5791297.594214105],
 [3.8696821515892403, 23075.924058679706],
 [0.3102974828375285, 1374.7036613272314],
 [52.160329962746154, 5094474.564977118],
 [51.26987984707823, 3467704.487747135],
 [40.49042769857448, 1567562.1950712814],
 [49.43961937716265, 1447919.230865052],
 [38.63936447711621, 5567298.330993614],
 [0.012711864406779662, 5.123728813559323],
 [36.79015232782662, 1454087.0379918504],
 [16.402175489485114, 1785801.5004205904],
 [1.353737214791505, 17080.3997482297

Identify the U.S. states that were “visited” by the travelers (NOTE:
We will adopt a loose definition of “visited”: any state traversed by the
journey is considered “visited” for purposes of this question.)

There were several natural landscape and man-made landmarks visited
on this trip (a landmark being defined loosely as a tourist destination).
Name three of them. (NOTE: To assist you with this task, it will
be helpful to also plot the locations of some landmarks on a map,
and see whether the traveler’s journey “intersected with” or otherwise
spent significant time near one of these landmarks. Once you identify
the two major cities visited on the journey, you will have sufficient
information to identify at least three landmarks. How? By Googling
the names of the two cities and making a list of some of the noteworthy
landmarks in each, and then, obtaining the GPS coordinates of these
landmarks!) Can you name additional landmarks, particularly outside
densely populated areas, that were “visited” in transit? One homework
extra credit point will be awarded for every five additional landmarks
identified, up to +3 homework extra credit points. (As an aside, and
FYI only, natural landscape landmarks–of which there are several on
this journey–are best “visited” at low speeds: If not, then it becomes
hard to photograph them!)

Create a graphically pleasant Powerpoint presentation of your above
analysis of the combined journey, using the template format outlined
in Homework 2. In this case, don’t consider the journey as a problem
to be solved. Instead, consider the journey as a product to be sold. That
is precisely why you were just asked to identify states and landmarks!
Your Powerpoint presentation should reflect this “sales” viewpoint.

Create a separate document (either Powerpoint or word-processed text
document) that is a technical appendix and that includes any and all
python code that you write.